# Solveur de Jacobi : Modèles de programmation Multi-GPU
Ce notebook présente 12 versions progressives d’un solveur de Jacobi 2D. Chaque section explique le modèle ou l’optimisation, compile la version, exécute et collecte les métriques Nsight.

## etape1_cpu
**Description :** Solveur Jacobi CPU de base : implémentation mono-thread. Utile pour valider la correction et les petites tailles de problème ; met en évidence la limite de calcul CPU.

**Intérêt :** Baseline : évalue la limite mémoire CPU pour établir une référence.

In [ ]:
%%bash cd etape1_cpu ; make all

In [ ]:
%%bash cd etape1_cpu ; nv-nsight-cu-cli --csv --report-file rapport_etape1_cpu.csv ./main ;cat rapport_etape1_cpu.csv

## etape2_cpu_gpu
**Description :** 1 CPU + 1 GPU : délégation du noyau Jacobi à un GPU. Montre l'accélération GPU quand le problème tient sur un seul dispositif.

**Intérêt :** Met en évidence l'écart de performance CPU vs GPU lorsque la grille est suffisamment grande.

In [ ]:
%%bash cd etape2_cpu_gpu ; make all

In [ ]:
%%bash cd etape2_cpu_gpu ; nv-nsight-cu-cli --csv --report-file rapport_etape2_cpu_gpu.csv ./main ;cat rapport_etape2_cpu_gpu.csv

## etape3_mpi_gpus
**Description :** MPI + GPUs : domaine réparti sur plusieurs rangs MPI, chacun pilotant un GPU. Illustrations des défis de mise à l'échelle multi-nœuds et des communications inter-rangs.

**Intérêt :** Test de montée en charge inter-nœuds et coût MPI sur cluster multi-GPU.

In [ ]:
%%bash cd etape3_mpi_gpus ; make all

In [ ]:
%%bash cd etape3_mpi_gpus ; nv-nsight-cu-cli --csv --report-file rapport_etape3_mpi_gpus.csv ./main ;cat rapport_etape3_mpi_gpus.csv

## etape4_mpi_overlap
**Description :** MPI + recouvrement : recouvrements des échanges d’halo non-bloquants avec le calcul Jacobi local. Réduit l'impact de la latence réseau.

**Intérêt :** Cache la latence réseau en recouvrant communication et calcul local.

In [ ]:
%%bash cd etape4_mpi_overlap ; make all

In [ ]:
%%bash cd etape4_mpi_overlap ; nv-nsight-cu-cli --csv --report-file rapport_etape4_mpi_overlap.csv ./main ;cat rapport_etape4_mpi_overlap.csv

## etape5_nccl
**Description :** NCCL : utilisation de la NVIDIA Collective Communications Library pour les échanges GPU à GPU. Montre les gains via NVLink ou PCIe haute bande passante.

**Intérêt :** Exploite automatiquement le topologie NVLink/PCIe pour des échanges GPU efficaces.

In [ ]:
%%bash cd etape5_nccl ; make all

In [ ]:
%%bash cd etape5_nccl ; nv-nsight-cu-cli --csv --report-file rapport_etape5_nccl.csv ./main ;cat rapport_etape5_nccl.csv

## etape6_nccl_overlap
**Description :** NCCL + recouvrement : superposition des collectifs NCCL avec le calcul sur GPU, cachant le coût de communication.

**Intérêt :** Essentiel à forte densité GPU pour maintenir les cœurs occupés.

In [ ]:
%%bash cd etape6_nccl_overlap ; make all

In [ ]:
%%bash cd etape6_nccl_overlap ; nv-nsight-cu-cli --csv --report-file rapport_etape6_nccl_overlap.csv ./main ;cat rapport_etape6_nccl_overlap.csv

## etape7_nccl_graphs
**Description :** NCCL + CUDA Graphs : capture et relecture des séquences Jacobi/échange pour réduire le surcoût des lancements.

**Intérêt :** Réduit l’overhead de lancement grâce aux CUDA Graphs.

In [ ]:
%%bash cd etape7_nccl_graphs ; make all

In [ ]:
%%bash cd etape7_nccl_graphs ; nv-nsight-cu-cli --csv --report-file rapport_etape7_nccl_graphs.csv ./main ;cat rapport_etape7_nccl_graphs.csv

## etape8_nvshmem
**Description :** NVSHMEM : modèle PGAS à accès mémoire unilatéral GPU, simplifiant les mises à jour d’halo.

**Intérêt :** Simplifie les échanges via modèle PGAS unilatéral.

In [ ]:
%%bash cd etape8_nvshmem ; make all

In [ ]:
%%bash cd etape8_nvshmem ; nv-nsight-cu-cli --csv --report-file rapport_etape8_nvshmem.csv ./main ;cat rapport_etape8_nvshmem.csv

## etape9_nvshmem_lt
**Description :** NVSHMEM + LTO : ajout de l’optimisation link-time pour inliner les fonctions critiques et réduire le coût des appels.

**Intérêt :** Optimisation link-time pour inliner les sections critiques.

In [ ]:
%%bash cd etape9_nvshmem_lt ; make all

In [ ]:
%%bash cd etape9_nvshmem_lt ; nv-nsight-cu-cli --csv --report-file rapport_etape9_nvshmem_lt.csv ./main ;cat rapport_etape9_nvshmem_lt.csv

## etape10_vshmem_neighborhood_lto
**Description :** vshmem neighborhood_sync + LTO : synchronisation fine-grain de voisinage et optimisations link-time O2.

**Intérêt :** Synchronisation fine et LTO pour boucles serrées.

In [ ]:
%%bash cd etape10_vshmem_neighborhood_lto ; make all

In [ ]:
%%bash cd etape10_vshmem_neighborhood_lto ; nv-nsight-cu-cli --csv --report-file rapport_etape10_vshmem_neighborhood_lto.csv ./main ;cat rapport_etape10_vshmem_neighborhood_lto.csv

## etape11_nvshmem_norm_overlap_neighborhood_sync_lto
**Description :** Combinaison : NVSHMEM avec recouvrement, synchrone de voisinage, et LTO pour maximiser la concurrence.

**Intérêt :** Combinaison des meilleures pratiques pour un binaire ultra-optimisé.

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto ; make all

In [ ]:
%%bash cd etape11_nvshmem_norm_overlap_neighborhood_sync_lto ; nv-nsight-cu-cli --csv --report-file rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv ./main ;cat rapport_etape11_nvshmem_norm_overlap_neighborhood_sync_lto.csv

## etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1
**Description :** Tuning étendu : paramètres ajustables (taille de tuile, ordre de boucles) et hooks de benchmark.

**Intérêt :** Ajout de paramètres de tuning et hooks de benchmarking.

In [ ]:
%%bash cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 ; make all

UsageError: Line magic function `%bash` not found (But cell magic `%%bash` exists, did you mean that instead?).


In [ ]:
%%bash cd etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1 ; nv-nsight-cu-cli --csv --report-file rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv ./main ;cat rapport_etape12_nvshmem_norm_overlap_neighborhood_sync_lto_ext1.csv